In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent
import os,sys
from sklearn.metrics import precision_recall_curve,confusion_matrix,precision_score,recall_score,f1_score
from tqdm.notebook import tqdm

In [5]:
def get_files_scan_one_cpu(dir):
    """Get the path of all the files in nested folders"""
    all_files = []
    for entry in os.scandir(dir):
        if entry.is_file():
            all_files.append(entry.path)
        elif entry.is_dir():
            all_files.extend(get_files_scan(entry.path))
    if len(all_files) > 0:
        return all_files
    else:
        raise FileNotFoundError("No Files found. Please check your directory.")

def scan_files_in_folder(folder):
    """Scans files in a folder and returns a list of file paths."""
    files = []
    for entry in os.scandir(folder):
        if entry.is_file():
            files.append(entry.path)
        elif entry.is_dir():
            files.extend(scan_files_in_folder(entry.path))
    return files

def get_files_scan(dir,max_workers=1):
    """Get the path of all the files in nested folders using concurrent.futures."""
    all_files = []
    if max_workers==1:
        return get_files_scan_one_cpu(dir)
    else:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_dir = {executor.submit(scan_files_in_folder, entry.path): entry.path for entry in os.scandir(dir)}
            for future in concurrent.futures.as_completed(future_to_dir):
                folder = future_to_dir[future]
                try:
                    all_files.extend(future.result())
                except Exception as e:
                    print(f"An error occurred while scanning {folder}: {e}")

        if len(all_files) > 0:
            return all_files
        else:
            raise FileNotFoundError("No Files found. Please check your directory.")

In [6]:
CC3D_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_10/"
ABC_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_8/"

# CC3D - Boundary (Model Trained on CC3D and Tested on CC3D)

In [7]:
cc3d_bndry_gt_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","gt"),4)
cc3d_bndry_pred_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","pred"),4)

In [8]:
all_precision_scores=[]
all_recall_scores=[]
for i in tqdm(range(len(cc3d_bndry_gt_files))):
    gt_bndry=np.load(cc3d_bndry_gt_files[i])
    pred_bndry=np.load("/".join(cc3d_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                cc3d_bndry_gt_files[i].split("/")[-3:]))
    pred_class=(pred_bndry>0.56).astype(int)
    all_precision_scores.append(precision_score(gt_bndry,pred_class))
    all_recall_scores.append(recall_score(gt_bndry,pred_class))

  0%|          | 0/4609 [00:00<?, ?it/s]

In [12]:
print(f"CC3D Boundary Detection Precision: {np.mean(all_precision_scores)}")
print(f"CC3D Boundary Detection Recall: {np.mean(all_recall_scores)}")

CC3D Boundary Detection Precision: 0.6517300129252261
CC3D Boundary Detection Recall: 0.6965228597627664


# CC3D - JNC (Model Trained on CC3D and Tested on CC3D)

In [13]:
cc3d_jnc_gt_files=get_files_scan(os.path.join("/mnt/isilon/mkhan/scan2brep/test/BRepJd/version_2/","jnc","gt"),4)
cc3d_jnc_pred_files=get_files_scan(os.path.join("/mnt/isilon/mkhan/scan2brep/test/BRepJd/version_2/","jnc","pred"),4)

In [14]:
all_precision_scores_jnc=[]
all_recall_scores_jnc=[]
all_f1_scores_jnc=[]
for i in tqdm(range(len(cc3d_jnc_gt_files))):
    gt_jnc=np.load(cc3d_jnc_gt_files[i])
    pred_jnc=np.load("/".join(cc3d_jnc_gt_files[i].split("/")[:-4]+["pred"]+\
                                cc3d_jnc_gt_files[i].split("/")[-3:]))
    pred_class=(pred_jnc>0.48).astype(int)
    all_precision_scores_jnc.append(precision_score(gt_jnc,pred_class))
    all_recall_scores_jnc.append(recall_score(gt_jnc,pred_class))
    all_f1_scores_jnc.append(f1_score(gt_jnc,pred_class))

  0%|          | 0/4609 [00:00<?, ?it/s]

/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages

In [21]:
print(f"CC3D Junction Detection Precision: {np.mean(all_precision_scores_jnc)}")
print(f"CC3D Junction Detection Recall: {np.mean(all_recall_scores_jnc)}")

CC3D Junction Detection Precision: 0.42735551238813363
CC3D Junction Detection Recall: 0.36410591244602325


# ABC - BNDRY (Model Trained on ABC and Tested on ABC)

In [16]:
from torchmetrics.classification import PrecisionRecallCurve,BinaryPrecisionRecallCurve
import torch

In [22]:
abc_bndry_gt_files=get_files_scan(os.path.join(ABC_BDN_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(ABC_BDN_PATH,"bndry","pred"),4)

In [23]:
# all_gt_bndry=[]
# all_pred_bndry=[]
# for i in tqdm(range(len(abc_bndry_gt_files))):
#     gt_bndry=np.load(abc_bndry_gt_files[i])
#     pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
#                                 abc_bndry_gt_files[i].split("/")[-3:]))
    
#     all_gt_bndry.append(list(gt_bndry))
#     all_pred_bndry.append(list(pred_bndry))

In [24]:
all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.56).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/955 [00:00<?, ?it/s]

In [25]:
# all_precision_scores_abc_bndry=[]
# all_recall_scores_abc_bndry=[]
# for i in range(len(abc_bndry_gt_files)):
#     gt_bndry=np.load(abc_bndry_gt_files[i])
#     pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
#                                 abc_bndry_gt_files[i].split("/")[-3:]))
    
#     all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
#     all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))

In [26]:
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry)}")
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry)}")

ABC Boundary Detection Precision: 0.49577775994984297
ABC Boundary Detection Recall: 0.6306113410641406


# ABC - JNC (Model Trained on ABC and Tested on ABC)

In [43]:
abc_jnc_gt_files=get_files_scan(os.path.join("/mnt/isilon/mkhan/scan2brep/test/BRepJd/version_1/","jnc","gt"),4)
abc_jnc_pred_files=get_files_scan(os.path.join("/mnt/isilon/mkhan/scan2brep/test/BRepJd/version_1/","jnc","pred"),4)

In [28]:
# precision_scores=[]
# recall_scores=[]
# pr_curve = PrecisionRecallCurve(task="binary",thresholds=100)
# for i in tqdm(range(len(abc_jnc_gt_files))):
#     gt_jnc=np.load(abc_jnc_gt_files[i])
#     pred_jnc=np.load("/".join(abc_jnc_gt_files[i].split("/")[:-4]+["pred"]+\
#                                 abc_jnc_gt_files[i].split("/")[-3:]))
    
#     # all_gt_jnc.append(list(gt_jnc))
#     # all_pred_jnc.append(list(pred_jnc))
#     precision, recall, thresholds = pr_curve(1-torch.from_numpy(pred_jnc), 
#                                              torch.from_numpy(gt_jnc))
    
#     precision_scores.append(precision)
#     recall_scores.append(recall)

In [29]:
# pr_df=pd.DataFrame({"precision":torch.stack(precision_scores).mean(axis=0)[1:],
#                     "recall":torch.stack(recall_scores).mean(axis=0)[1:],"thresholds":thresholds})

In [47]:
all_precision_scores_abc_jnc=[]
all_recall_scores_abc_jnc=[]
all_f1_scores_abc_jnc=[]
for i in tqdm(range(len(abc_jnc_gt_files))):
    gt_jnc=np.load(abc_jnc_gt_files[i])
    pred_jnc=np.load("/".join(abc_jnc_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_jnc_gt_files[i].split("/")[-3:]))
    pred_jnc=(pred_jnc>0.55).astype(int)
    all_precision_scores_abc_jnc.append(precision_score(gt_jnc,pred_jnc))
    all_recall_scores_abc_jnc.append(recall_score(gt_jnc,pred_jnc))
    all_f1_scores_abc_jnc.append(f1_score(gt_jnc,pred_jnc))

  0%|          | 0/4609 [00:00<?, ?it/s]

/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages

In [46]:
print(f"ABC Junction Detection Precision: {np.mean(all_precision_scores_abc_jnc)}")
print(f"ABC Junction Detection Recall: {np.mean(all_recall_scores_abc_jnc)}")

ABC Junction Detection Precision: 0.33195286679192537
ABC Junction Detection Recall: 0.6258307234947242


# Summary

In [42]:
print("CC3D (Model Trained on CC3D)")
print("   Boundary-->")
print(f"     Precision: {np.mean(all_precision_scores).round(3)}")
print(f"     Recall: {np.mean(all_recall_scores).round(3)}")
print("   Junction-->")
print(f"     Precision: {np.mean(all_precision_scores_jnc).round(3)}")
print(f"     Recall: {np.mean(all_recall_scores_jnc).round(3)}")
print("\n")
print("ABC (Model Trained on ABC)")
print("   Boundary-->")
print(f"     Precision: {np.mean(all_precision_scores_abc_bndry).round(3)}")
print(f"     Recall: {np.mean(all_recall_scores_abc_bndry).round(3)}")
print("   Junction-->")
print(f"     Precision: {np.mean(all_precision_scores_abc_jnc).round(3)}")
print(f"     Recall: {np.mean(all_recall_scores_abc_jnc).round(3)}")

CC3D (Model Trained on CC3D)
   Boundary-->
     Precision: 0.652
     Recall: 0.697
   Junction-->
     Precision: 0.427
     Recall: 0.364


ABC (Model Trained on ABC)
   Boundary-->
     Precision: 0.496
     Recall: 0.631
   Junction-->
     Precision: 0.273
     Recall: 0.361
